In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import time
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/jz2308/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
tcal=gtfs.TransitCalendar('2015-12-03')
thurs_services = tcal.get_service_ids('2015-12-03')
thurs_trips = trips.service_id.isin(thurs_services)
thurs_stops = stop_times.reset_index().set_index('trip_id').loc[thurs_trips]
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
interpolated = pd.read_csv('2015-12-03_interpolated.csv')

In [4]:
interpolated.head()

,route,trip_id,trip_date,vehicle_id,stop_id,interpolated_arrival_time
0,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402805,0 days 06:10:56.778916
1,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,405057,0 days 06:14:24.977703
2,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,403416,0 days 06:16:58.526210
3,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402809,0 days 06:17:25.226045
4,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,402811,0 days 06:18:36.819142


In [5]:
interpolated['interpolated_arrival_time'] = pd.to_timedelta(interpolated['interpolated_arrival_time'])

In [6]:
interpolated = interpolated.merge(stop_times[['arrival_time','stop_sequence']],how='left',left_on=['trip_id','stop_id'],right_index=True)

In [7]:
# bools = interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=False) + interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=True)
# interpolated[bools].sort(['route','trip_id','trip_date','vehicle_id']).shape

In [8]:
interpolated.set_index(['route','trip_id','trip_date','vehicle_id','stop_sequence','stop_id'],inplace=True)

## Get the densest stop_id by shape (trip?)

In [14]:
trip_groups = interpolated.groupby(level=(0,2,5,4)).size()

In [22]:
def series_max(s):
    return s.idxmax()[2]

In [23]:
densest_stops = trip_groups.groupby(level=(0)).apply(series_max)
densest_stops

route
MTA NYCT_B1      300009
MTA NYCT_B11     301173
MTA NYCT_B12     301328
MTA NYCT_B13     301505
MTA NYCT_B14     301598
MTA NYCT_B15     301135
MTA NYCT_B16     301677
MTA NYCT_B17     300853
MTA NYCT_B2      300173
MTA NYCT_B20     301959
MTA NYCT_B24     503993
MTA NYCT_B25     302383
MTA NYCT_B26     302372
MTA NYCT_B3      300261
MTA NYCT_B31     300175
MTA NYCT_B32     302304
MTA NYCT_B35     302692
MTA NYCT_B36     300019
MTA NYCT_B37     302914
MTA NYCT_B38     302948
MTA NYCT_B39     403679
MTA NYCT_B4      300368
MTA NYCT_B41     303295
MTA NYCT_B42     303358
MTA NYCT_B43     303711
MTA NYCT_B44     303483
MTA NYCT_B44+    303441
MTA NYCT_B45     303531
MTA NYCT_B46     303592
MTA NYCT_B47     300857
                  ...  
MTABC_Q50        101743
MTABC_Q52        550070
MTABC_Q53        550070
MTABC_Q60        504399
MTABC_Q64        551812
MTABC_Q65        550032
MTABC_Q66        551894
MTABC_Q67        503832
MTABC_Q69        550010
MTABC_Q70        504542
MTABC_Q72 

## Function that can applied repeatedly

In [24]:
# convert scheduled time to numeric
interpolated['schedule_convert'] = pd.to_timedelta(interpolated['arrival_time'])

In [25]:
def peak_hour(x):
    if x > '06:00:00' and x < '09:00:00':
        return True
    else:
        if x > '16:00:00' and x < '19:00:00':
            return True
        else:
            return False

In [26]:
def wait_ass(x,y):
    if x == True:
        if y >= 3*60:
            return True
        else:
            return False
    if x == False:
        if y>= 5*60:
            return True
        else:
            return False         

In [63]:
def wa_at_stop(route,stop):
    pd.options.mode.chained_assignment = None
    interpolated_2 = interpolated.xs((route,stop),level=(0,5),drop_level=False)
    interpolated_2.reset_index(inplace=True)
    interpolated_2['sche_headway'] = interpolated_2.sort(columns='schedule_convert')['schedule_convert'].diff()
    interpolated_2['inter_headway'] = interpolated_2.sort(columns='interpolated_arrival_time')['interpolated_arrival_time'].diff()
    interpolated_2['sche_h_sec'] = interpolated_2['sche_headway']/ttools.datetime.timedelta(seconds=1)
    interpolated_2['inter_h_sec'] = interpolated_2['inter_headway']/ttools.datetime.timedelta(seconds=1)
    interpolated_2['P_hour'] = map(lambda x:peak_hour(x),interpolated_2['arrival_time'])
    interpolated_2['diff'] = interpolated_2['inter_h_sec'] - interpolated_2['sche_h_sec']
    interpolated_2['wait_ass'] = map(lambda x,y:wait_ass(x,y), interpolated_2['P_hour'],interpolated_2['diff'])
    ##wait assessment for the single route on the single stop
    # 100% is best
    return 1.0-(1.0*interpolated_2['wait_ass'].sum()/len(interpolated_2['wait_ass']))

In [64]:
interpolated.head()

interpolated_arrival_time  \
route         trip_id                           trip_date  vehicle_id    stop_sequence stop_id                              
MTA NYCT_M101 OH_D5-Weekday-SDon-036500_M101_38 2015-12-03 MTA NYCT_5879 2             402805             06:10:56.778916   
                                                                         3             405057             06:14:24.977703   
                                                                         4             403416             06:16:58.526210   
                                                                         5             402809             06:17:25.226045   
                                                                         6             402811             06:18:36.819142   

                                                                                               arrival_time  \
route         trip_id                           trip_date  vehicle_id    stop_sequence stop_id                
MTA NYCT_M101 OH_D5-Weekday-SDon-036500_M101_38 2015-12-03 MTA NYCT_5879 2             402805      06:06:17   
                                                                         3             405057      06:07:38   
                                                                         4             403416      06:09:06   
                                                                         5             402809      06:09:48   
                                                                         6             402811      06:11:41   

                                                                                                schedule_convert  
route         trip_id                           trip_date  vehicle_id    stop_sequence stop_id                    
MTA NYCT_M101 OH_D5-Weekday-SDon-036500_M101_38 2015-12-03 MTA NYCT_5879 2             402805           06:06:17  
                                                                         3             405057           06:07:38  
                                                                         4             403416           06:09:06  
                                                                         5             402809           06:09:48  
                                                                         6             402811           06:11:41

In [65]:
wa_at_stop('MTA NYCT_B1',300009)

0.8957055214723927

In [66]:
all_routes = pd.Series(index=densest_stops.index)
for i,v in densest_stops.iteritems():
    all_routes[i] = wa_at_stop(i,v)

In [67]:
all_routes.mean()

0.79945940207255217

In [74]:
all_stops = pd.Series(index=trip_groups.index)
for i, v in trip_groups.iteritems():
    if v > 50:
        all_stops[i] = wa_at_stop(i[0],i[2])

In [93]:
all_stops_df = pd.DataFrame.from_dict(all_stops).dropna().reset_index().rename(columns={0:'wa'})

In [94]:
all_stops_df.corr()

,stop_id,stop_sequence,wa
stop_id,1.000000,-0.014718,-0.090302
stop_sequence,-0.014718,1.000000,-0.538366
wa,-0.090302,-0.538366,1.000000


In [95]:
import statsmodels.formula.api as smf
results = smf.ols('wa ~ stop_sequence', data=all_stops_df.dropna().reset_index()).fit()
print results.summary()

                            OLS Regression Results                            
Dep. Variable:                     wa   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     6701.
Date:                Mon, 18 Jul 2016   Prob (F-statistic):               0.00
Time:                        11:05:40   Log-Likelihood:                 25037.
No. Observations:               16421   AIC:                        -5.007e+04
Df Residuals:                   16419   BIC:                        -5.005e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept         0.8405      0.001   1141.083

In [99]:
def mean_hw_dev_at_stop(route,stop):
    pd.options.mode.chained_assignment = None
    interpolated_2 = interpolated.xs((route,stop),level=(0,5),drop_level=False)
    interpolated_2.reset_index(inplace=True)
    interpolated_2['sche_headway'] = interpolated_2.sort(columns='schedule_convert')['schedule_convert'].diff()
    interpolated_2['inter_headway'] = interpolated_2.sort(columns='interpolated_arrival_time')['interpolated_arrival_time'].diff()
    interpolated_2['sche_h_sec'] = interpolated_2['sche_headway']/ttools.datetime.timedelta(seconds=1)
    interpolated_2['inter_h_sec'] = interpolated_2['inter_headway']/ttools.datetime.timedelta(seconds=1)
    # interpolated_2['P_hour'] = map(lambda x:peak_hour(x),interpolated_2['arrival_time'])
    interpolated_2['abs_diff'] = abs(interpolated_2['inter_h_sec'] - interpolated_2['sche_h_sec'])
    # interpolated_2['wait_ass'] = map(lambda x,y:wait_ass(x,y), interpolated_2['P_hour'],interpolated_2['diff'])
    ##wait assessment for the single route on the single stop
    # 100% is best
    return interpolated_2['abs_diff'].dropna().mean()

In [100]:
mean_hw_dev_at_stop('MTA NYCT_B1',300009)

136.72597633950616

In [101]:
all_routes_hwdev = pd.Series(index=densest_stops.index)
for i,v in densest_stops.iteritems():
    all_routes_hwdev[i] = mean_hw_dev_at_stop(i,v)

In [102]:
all_routes_hwdev.mean()

292.6473991789245

In [103]:
all_stops_hwdev = pd.Series(index=trip_groups.index)
for i, v in trip_groups.iteritems():
    if v > 50:
        all_stops_hwdev[i] = mean_hw_dev_at_stop(i[0],i[2])

In [104]:
all_stops_hwdev_df = pd.DataFrame.from_dict(all_stops_hwdev).dropna().reset_index().rename(columns={0:'mean_hwdev'})
results = smf.ols('mean_hwdev ~ stop_sequence', data=all_stops_hwdev_df.dropna().reset_index()).fit()
print results.summary()

                            OLS Regression Results                            
Dep. Variable:             mean_hwdev   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     5383.
Date:                Mon, 18 Jul 2016   Prob (F-statistic):               0.00
Time:                        11:32:20   Log-Likelihood:                -97228.
No. Observations:               16421   AIC:                         1.945e+05
Df Residuals:                   16419   BIC:                         1.945e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept       231.7930      1.261    183.775